# Third Project: Building the Abstract Syntax Tree

Abstract syntax trees are data structures that better represent the structure of the
program code than the parse tree. An AST can be edited and enhanced with information
such as properties and annotations for each element it contains. Your goal in this
third project is to transform the parse tree into an AST.

## Abstract Syntax Tree objects
We've shown how to use PLY to parse a grammar and generate a tree-like structure from it.
Tuples, however, are a bit limited. Let's upgrade our AST to classes.

Each node in our tree will be represented by a class.
Your job is to use the parser to connect these classes/nodes using the PLY parser to effectively buid the AST.

This section defines classes for different kinds of nodes of an Abstract Syntax Tree.
During parsing, you will create these nodes and connect them together. In general,
you will have a different AST node for each kind of grammar rule.

### Helpers

Below is a list of the helper methods/classes:

* `class Node` - Every nodes inherts this class. It holds some base functionality for all nodes.
* `class DeclType` - It holds some common functionalities for declaration nodes.
* `def represent_node` - Used to dump each node as a string.

These helpers don't deserve much atention since you won't be using then directly.

In [ ]:
def represent_node(obj, indent):
    def _repr(obj, indent, printed_set):
        """
        Get the representation of an object, with dedicated pprint-like format for lists.
        """
        if isinstance(obj, list):
            indent += 1
            sep = ",\n" + (" " * indent)
            final_sep = ",\n" + (" " * (indent - 1))
            return (
                "["
                + (sep.join((_repr(e, indent, printed_set) for e in obj)))
                + final_sep
                + "]"
            )
        elif isinstance(obj, Node):
            if obj in printed_set:
                return ""
            else:
                printed_set.add(obj)
            result = obj.__class__.__name__ + "("
            indent += len(obj.__class__.__name__) + 1
            attrs = []

            # convert each node attribute to string
            for name, value in vars(obj).items():

                # is an irrelevant attribute: skip it.
                if name in ('bind', 'coord'):
                    continue

                # relevant attribte not set: skip it.
                if value is None:
                    continue

                # relevant attribute set: append string representation.
                value_str = _repr(value, indent + len(name) + 1, printed_set)
                attrs.append(name + "=" + value_str)

            sep = ",\n" + (" " * indent)
            final_sep = ",\n" + (" " * (indent - 1))
            result += sep.join(attrs)
            result += ")"
            return result
        elif isinstance(obj, str):
            return obj
        else:
            return str(obj)

    # avoid infinite recursion with printed_set
    printed_set = set()
    return _repr(obj, indent, printed_set)


# ABSTRACT NODES

class Node(ABC):
    """Abstract base class for AST nodes."""

    attr_names = ()

    @abstractmethod
    def __init__(self, coord=None):
        self.coord = coord

    def __repr__(self):
        """Generates a python representation of the current node"""
        return represent_node(self, 0)

    def children(self):
        """A sequence of all children that are Nodes"""
        pass

    def show(
        self,
        buf=sys.stdout,
        offset=0,
        attrnames=False,
        nodenames=False,
        showcoord=False,
        _my_node_name=None,
    ):
        """Pretty print the Node and all its attributes and children (recursively) to a buffer.
        buf:
            Open IO buffer into which the Node is printed.
        offset:
            Initial offset (amount of leading spaces)
        attrnames:
            True if you want to see the attribute names in name=value pairs. False to only see the values.
        nodenames:
            True if you want to see the actual node names within their parents.
        showcoord:
            Do you want the coordinates of each Node to be displayed.
        """
        lead = " " * offset
        if nodenames and _my_node_name is not None:
            buf.write(lead + self.__class__.__name__ + " <" + _my_node_name + ">: ")
            inner_offset = len(self.__class__.__name__ + " <" + _my_node_name + ">: ")
        else:
            buf.write(lead + self.__class__.__name__ + ":")
            inner_offset = len(self.__class__.__name__ + ":")

        if self.attr_names:
            if attrnames:
                nvlist = [
                    (n, represent_node(getattr(self, n), offset+inner_offset+1+len(n)+1))
                    for n in self.attr_names
                    if getattr(self, n) is not None
                ]
                attrstr = ", ".join("%s=%s" % nv for nv in nvlist)
            else:
                vlist = [getattr(self, n) for n in self.attr_names]
                attrstr = ", ".join(
                    represent_node(v, offset + inner_offset + 1) for v in vlist
                )
            buf.write(" " + attrstr)

        if showcoord:
            if self.coord and self.coord.line != 0:
                buf.write(" %s" % self.coord)
        buf.write("\n")

        for (child_name, child) in self.children():
            child.show(buf, offset + 4, attrnames, nodenames, showcoord, child_name)


class DeclType(Node):
    """
    Absctract class for declaration types.

    This class augments declaration nodes with a few extra tools.

    :attribute identifier:
        Allows the user to set/get the declaration's ID node from any
        declaration in the declaration chain.

    :attribute primitive:
        Allows the user to set/get uderlying primitive type (int, char, etc.)
        from any declaration node in the declaration chain.

    :method modify:
        Used to apply a type modifier to a declaration chain.
    """

    @abstractmethod
    def __init__(self):
        ...

    @property
    def identifier(self):
        """Get the declaration's ID node."""
        # ID not set: return as empty.
        if self.type is None:
            return None

        # has type modififer: fetch ID from modifier.
        return self.type.identifier

    @identifier.setter
    def identifier(self, identifier):
        """Set a declaration's identifier."""
        self.type.identifier = identifier

    def modify(self, modifier):
        """
        Apply a type modifier to a declaration.

        :param modifier: declaration modifier AST node (ArrayDecl).

        :returns: modified declaration.
        """
        # has primitive or unset type: modify itself
        if self.type is None or isinstance(self.type, Type):
            modifier.type = self.type
            self.type = modifier
            return self

        # has underlying type modifier: modify underlying modifier chain.
        self.type = self.type.modify(modifier)

        # return itself modified.
        return self

    @property
    def primitive(self):
        """Get the declaration's primitive type."""
        # type not set: return as empty.
        if self.type is None:
            return None

        # reached primitive type: return it.
        if isinstance(self.type, Type):
            return self.type

        # has a declaration modifier: recurse into it.
        return self.type.primitive

    @primitive.setter
    def primitive(self, typeNode):
        """
        Set the declaration's underlying primitive type.

        :param typeNode: primitive type node to be set.
        """
        # reached missing or primitive type: set/overwrite it
        if self.type is None or isinstance(self.type, Type):
            self.type = typeNode
            return

        # has a declaration modifier: recurse into it.
        self.type.primitive = typeNode


### AST Nodes

With the helpers defined, we can proceed to define the AST nodes. Below, you have some examples:

In [ ]:
class Program(Node):
    __slots__ = ('gdecls', 'coord')

    def __init__(self, gdecls, coord=None):
        self.gdecls = gdecls
        self.coord = coord

    def children(self):
        nodelist = []
        for i, child in enumerate(self.gdecls or []):
            nodelist.append(("gdecls[%d]" % i, child))
        return tuple(nodelist)

    attr_names = ()

Just as another example, for a binary operator, you might store the operator, the left
expression, and the right expression like this:

In [ ]:
class BinaryOp(Node):
    __slots__ = ("op", "lvalue", "rvalue", "coord")

    def __init__(self, op, left, right, coord=None):
        self.op = op
        self.lvalue = left
        self.rvalue = right
        self.coord = coord

    def children(self):
        nodelist = []
        if self.lvalue is not None:
            nodelist.append(("lvalue", self.lvalue))
        if self.rvalue is not None:
            nodelist.append(("rvalue", self.rvalue))
        return tuple(nodelist)

    attr_names = ("op",)

For Constant objects, you might store the type and value, like this:

In [ ]:
class Constant(Node):
    __slots__ = ("type", "value", "coord")

    def __init__(self, type, value, coord=None):
        self.type = type
        self.value = value
        self.coord = coord

    def children(self):
        return ()

    attr_names = ("type", "value")

Each node has a set of attributes that represent one of two things:

- A intrinsic property of the node (like code position).
- Child nodes (like the list of `GlobalDecl` in the `Program` node)

Each node also has a `attr_names` class attribute. This is used to print the nodes.

**Just a reminder that these nodes are already defined in the project's template.

**Suggestion: You should start simple and incrementally work your way up to building the
complete grammar.

## Showing the AST

Consider the uC program example below. Whenever a reduction occurs in the parser, a semantic action is executed. With this knowledge we can return nodes that represent the reduction in question instead of simple tuples.

```
/* comment */
int j = 3;
int main () {
  int i = j;
  int k = 3;
  int p = 2 * j;
  assert p == 2 * i;
}
```

The ```show``` method of class ```Node``` is called to generate a textual representation of the AST nodes and print all its attributes. The dump of the AST for the example above looks like this:

```
Program:
    GlobalDecl:
        Decl: ID(name=j)
            VarDecl:
                Type: int @ 2:1
            Constant: int, 3 @ 2:9
    FuncDef:
        Type: int @ 3:1
        Decl: ID(name=main)
            FuncDecl:
                VarDecl:
                    Type: int @ 3:1
        Compound: @ 3:13
            Decl: ID(name=i)
                VarDecl:
                    Type: int @ 4:3
                ID: j @ 4:11
            Decl: ID(name=k)
                VarDecl:
                    Type: int @ 5:3
                Constant: int, 3 @ 5:11
            Decl: ID(name=p)
                VarDecl:
                    Type: int @ 6:3
                BinaryOp: * @ 6:11
                    Constant: int, 2 @ 6:11
                    ID: j @ 6:15
            Assert: @ 7:3
                BinaryOp: == @ 7:10
                    ID: p @ 7:10
                    BinaryOp: * @ 7:15
                        Constant: int, 2 @ 7:15
                        ID: i @ 7:19
```

## Dealing with line and column information in AST

In [ ]:
class Coord:
    """Coordinates of a syntactic element. Consists of:
    - Line number
    - (optional) column number, for the Lexer
    """

    def __init__(self, line, column=None):
        self.line = line
        self.column = column

    def __str__(self):
        if self.line and self.column is not None:
            coord_str = "@ %s:%s" % (self.line, self.column)
        elif self.line:
            coord_str = "@ %s" % (self.line)
        else:
            coord_str = ""
        return coord_str

The ```Coord``` class above should be used to store (and show in the AST) the lines and
columns of the productions in the source code. To capture the coordinates for a
production ```p``` of the parser indexed with ```token_idx``` use the following code
in the UCParser class (the coordinate includes the ```lineno``` and the ```column```.
Both follow the semantics of the lex, starting at 1):

In [ ]:
    def _token_coord(self, p, token_idx):
        last_cr = p.lexer.lexer.lexdata.rfind('\n', 0, p.lexpos(token_idx))
        if last_cr < 0:
            last_cr = -1
        column = (p.lexpos(token_idx) - (last_cr))
        return Coord(p.lineno(token_idx), column)

## Building Declarations

At uC, statements can come several in a line. For example:
```
  int x, y, z = 5;
```
However, there is a requeriment to build our AST: you must divide them to separate the Decl nodes, that is, you need to build a list of declarations all sharing the given specifiers."""

Due to the order in which declarators are constructed, they have to be fixed in order to
look like a normal AST.

When a declaration arrives from syntax construction, it has these problems:
 * The innermost VarDecl has no type (because the basic type is only known at the
   uppermost declaration level)
 * The declaration has no variable name since that is saved in the innermost VarDecl

The ```primitive``` method for ```DeclType``` helper class fixes these problems.

For instance, in the ```declaration``` production, first, you instantiate the basic type (e.g., int, char,). After, set this type in all declarators within the ```init_declarator_list``` production. Then, you can return the complete ```Decl``` nodes.

The AST for the statement: ```int x, y, z = 5;```look like this:
```
Program:
    GlobalDecl:
        Decl: ID(name=x)
            VarDecl:
                Type: int @ 1:1
        Decl: ID(name=y)
            VarDecl:
                Type: int @ 1:1
        Decl: ID(name=z)
            VarDecl:
                Type: int @ 1:1
            Constant: int, 5 @ 1:15
```

### Type Modifier

A uC type consists of a basic type declaration, with a list of modifiers. For example:
```  
int c[5];
```
The basic declaration here is ```int c```, and the ```array``` is the modifier.
Basic declarations are represented by **VarDecl** and the modifiers are **FuncDecl**
 and **ArrayDecl**.

The standard states that whenever a new modifier is parsed, it should be added to the
end of the list of modifiers. For example:

**Array Declarators**

In a declaration T D where D has the form D1 [constant_expression_opt] and the type of
the identifier in the declaration T D1 is "type-modifier T", the type of the identifier
of D is "type-modifier array of T"

### primitive & modify Helpers

The basic declaration represented by ```VarDecl``` holds the variable primitive type and name (```int c```).

However, when we instantiate ```VarDecl```, we have not yet parsed the variable primitive type. This happens because declaration lists share the same primitive type (int a, b, c;).
So the primitive type is only parsed after the entire declaration list is parsed.
At this point, if we wish to set the primitive type, we need go down all the modifiers until we reach the ```VarDecl``` node.

This is were the ```DeclType.primitive``` helper comes in handy: it writes/reads the type attribute from the underlying ```VarDecl``` node.

The ```DeclType.primitive``` attribute can be used anywhere in the declaration (Decl, ArrayDecl, etc.).

We also mentioned that the modifier must always be inserted at the end of the modifier list.
This is what the ```DeclType.modify``` does for you: it inserts the modifier right before the VarDecl node.

The AST for the statement ```int c[5];``` look like this:

```
Program:
    GlobalDecl:
        Decl: ID(name=c)
            ArrayDecl:
                VarDecl:
                    Type: int @ 1:1
                Constant: int, 5 @ 1:7
```


Remenber that, because of how the grammar works, array dimensions closer to the variable's name are parsed earlier. The AST for the statment ```int b[1][2];``` look like this:

```
Program:
    GlobalDecl:
        Decl: ID(name=b)
            ArrayDecl:
                ArrayDecl:
                    VarDecl:
                        Type: int @ 1:1
                    Constant: int, 2 @ 1:10
                Constant: int, 1 @ 1:7
```

### identifier Helper

The basic declaration represented by `VarDecl` holds the variable primitive type and name (`int b`). However, it is very usefull to access the declaraion name direcly from the `Decl` node.

This simplifies future projects that will require symbol tables.

The `DeclType.identifier` helper will go down to `VarDecl` and fetch its `declname` attribute.

## Building Function Definitions

Declarations always come as lists (because they can be several in one line); therefore,
a design decision was to wrap the function definition into a list as well, to make the
return value of global_declaration homogeneous. On the other hand, we do not embed
a ```FuncDef``` object inside the ```GlobalDecl``` class. So we will have:

In [ ]:
    def p_program(self, p):
        """ program  : global_declaration_list
        """
        p[0] = Program(p[1])

    def p_global_declaration_list(self, p):
        """ global_declaration_list : global_declaration
                                    | global_declaration_list global_declaration
        """
        p[0] = [p[1]] if len(p) == 2 else p[1] + [p[2]]

    def p_global_declaration_1(self, p):
        """ global_declaration    : declaration
        """
        p[0] = GlobalDecl(p[1])    // GlobalDecl is a AST class

    def p_global_declaration_2(self, p):
        """ global_declaration    : function_definition
        """
        p[0] = p[1]      # Note that FuncDef was not embedded into a GlobalDecl

No exemplo:
```
int i, j;
int main() {}
```
A seguinte AST é produzida:
```
Program:
    GlobalDecl:
        Decl: ID(name=i)
            VarDecl:
                Type: int @ 1:1
        Decl: ID(name=j)
            VarDecl:
                Type: int @ 1:1
    FuncDef:
        Type: int @ 2:1
        Decl: ID(name=main)
            FuncDecl:
                VarDecl:
                    Type: int @ 2:1
        ...
 ```

## Single Expressions

Expressions in the uC grammar can be seen as lists but can also appear as single
expression. In the latter case, in order not to "pollute" AST, a design decision was to
treat single expression differently. See the example for the print statement below:

```
char mc[] = "Susy";

int main(){
    print("Hello World!");
    print("Hello", mc, ". Welcome to MC921");
    return;
}

Program:
    GlobalDecl:
        Decl: ID(name=mc)
            ArrayDecl:
                VarDecl:
                    Type: char @ 1:1
            Constant: string, Susy @ 1:13
    FuncDef:
        Type: int @ 3:1
        Decl: ID(name=main)
            FuncDecl:
                VarDecl:
                    Type: int @ 3:1
        Compound: @ 3:11
            Print: @ 4:5
                Constant: string, Hello World! @ 4:11
            Print: @ 5:5
                ExprList: @ 5:11
                    Constant: string, Hello @ 5:11
                    ID: mc @ 5:20
                    Constant: string, . Welcome to MC921 @ 5:24
            Return: @ 6:5
```

To do this, we just write the code in the expression production:

In [ ]:
    def p_expression(self, p):
        """ expression  : assignment_expression
                        | expression COMMA assignment_expression
        """
        if len(p) == 2:   # single expression
            p[0] = p[1]
        else:
            if not isinstance(p[1], ExprList):
                p[1] = ExprList([p[1]])

            p[1].exprs.append(p[3])
            p[0] = p[1]